<div id='firstRun'>
    <h2 style="color:red">If this is your first time 
    opening the script or you do not see the user interface buttons, please click "Cell" then "Run All" in 
    the menu bar above.</h2>
</div>

In [192]:
import os, sys
import pandas as pd, ipywidgets as widgets
from IPython.display import display, HTML

In [193]:
%%html
<style>
.spec_button {
    font-size:2em;
}
#error {
    height: 60px;
    text-align: center;
    font-size:3em;
    font-weight: bold;
    color: red;
    padding: 5px;
}
</style>
<script>
code_show=true;
var x = document.getElementById("firstRun");
function code_toggle() {
    if (code_show){
        $('div.input').hide();
    } else {
        $('div.input').show();
    }
    x.style.display = "none";
    code_show = !code_show
} 
function changeColor(id)
{
  document.getElementById(id).style.color = "#27b307"; // forecolor
}
$( document ).ready(code_toggle);
</script>
The raw code for this notebook is hidden by default for easier reading.
To toggle the raw code on/off, click <a href="javascript:code_toggle()">here</a>.

In [194]:
#Create button widget which opens spec.csv in excel.
button = widgets.Button(description="Open Specification File", button_style = 'info', layout=widgets.Layout(width='auto',height = '50px'))
output = widgets.Output()
button.add_class('spec_button')

display(button, output)

def on_button_clicked(b):
    with output:
        print("Opening specification file...")
        os.system('start "excel" "spec.xlsx"')

button.on_click(on_button_clicked)

In [195]:
#Read spec.xlsx
try:
    spec = pd.read_excel('spec.xlsx', 'Spec')
except PermissionError as e:
    display(HTML('<p id="error">Close spec in excel</p>'))
    sys.exit(e)
out = pd.DataFrame(columns=['Generated Links'])
srcMode = spec.loc[0, 'Source Mode']
targMode = spec.loc[0, 'Target Mode']
id_counter = 0

In [196]:
#Format to make links clickable in dataframe display
def make_clickable(val):
    global id_counter
    name = val.split('#')[0]
    url = val.split('#')[1]
    id_counter += 1
    return '<a target="_blank" id="link{}" href="{}" onclick="changeColor({});">{}</a>'.format(id_counter, url, "'link"+str(id_counter)+"'", name)

In [197]:
#Main loop to generate links from Stat Act names and modes
for r, statAct in enumerate(spec['Stat Acts'].dropna().tolist()):
    statActURL='DTOstatisticalactivity.StatisticalActivityPK-name%3A'\
            +statAct.replace(' ', '%20').replace('&','%26')
    url = 'http://cordwl-p-02:7001/cordprd/cosmosna/runReports.do?'\
        'methodToCall=confirmComparisonReport&statactid='+statActURL+\
        '&m='+srcMode+'&id='+statActURL+'&comparisonModeName='+targMode
    out.loc[r+1, 'Generated Links'] = statAct+'#'+url

In [198]:
display(out.style.format({'Generated Links': make_clickable}))

,Generated Links
1,Balance of Payments Geographic
2,Balanced TiS Satellite
3,TiS Derived Outputs Satellite
4,TiS Geographic
5,TIS Sources INT and EXT
6,TIS Sources SURVEYS
7,TIS Sources TRANSPORT
8,TiS Unbalanced Satellite
9,UK TiG Balanced
10,UK TiG Constrained
